In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Checking Out the Data Statistically

In [ ]:
housing = pd.read_csv('/kaggle/input/usa-housing/USA_Housing.csv')
housing.head()

In [ ]:
housing.info()

In [ ]:
# Shape/Size of the data

housing.shape

In [ ]:
housing.describe()

- lets see if there is any null/missing values in the datasets or not. It's important to remove or
- replace all the missing values before moving further.

In [ ]:
housing.isnull()

- Checking different column names

In [ ]:
housing.columns

In [ ]:
housing.isnull().sum()

So there is no missing or null value in the dataset so far, hence we can proceed further hassle free.
Till now, we had a detailed look at the given data and fortunately we don't have any missing values. So, the data cleaning is not required for this data.

# Exploratory Data Analysis (EDA)
Let's create some simple plots to check out the data!

In [ ]:
sns.pairplot(housing)

As we can see here in the last line of graph that all the features seems to be in a linear relationship with price except Avg. Area Number of Bedroom.
We can also see this by plotting a separate graph

Now, let's understand the correlation between variable by plotting correlation plot.

In [ ]:
housing.corr()

Visualizing the above correlaton using a correlation matrix in heatmap

In [ ]:
sns.heatmap(housing.corr(),annot=True)

As we can see that price is more correlated to Avg. Income Area, House Age and Area Population than Number of Bedrooms and Rooms. Lets see these metrics in tabular format.

In [ ]:
# visualizing/checking the linear relationship between Price and Avg. Area Income by plotting their scatter Plot as below:
plt.scatter(housing.Price, housing[['Avg. Area Income']])

Now we gotta check the distribution of our target variable that is Price as below:

In [ ]:
sns.distplot(housing['Price'])

We can see the price plot seems like a bell shaped curve and all the price is normally distributed.

# Training a Linear Regression Model
Let's now begin to train out regression model! We will need to first split up our data into an X array that contains the features to train on, and a y array with the target variable, in this case the Price column. We will toss out the Address column because it only has text info that the linear regression model can't use.

# X and y arrays

In [ ]:
X = housing[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
               'Avg. Area Number of Bedrooms', 'Area Population']]
y = housing['Price']

# Train Test Split
Now let's split the data into a training set and a testing set. We will train out model on the training set and then use the test set to evaluate the model.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred_score = cross_val_score(model, X, y, cv=10)
    return pred_score.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

# Preparing Data For Linear Regression
Linear regression is been studied at great length, and there is a lot of literature on how your data must be structured to make best use of the model.

As such, there is a lot of sophistication when talking about these requirements and expectations which can be intimidating. In practice, you can uses these rules more as rules of thumb when using Ordinary Least Squares Regression, the most common implementation of linear regression.

Try different preparations of your data using these heuristics and see what works best for your problem.

-Linear Assumption. Linear regression assumes that the relationship between your input and output is linear. It does not support anything else. This may be obvious, but it is good to remember when you have a lot of attributes. You may need to transform data to make the relationship linear (e.g. log transform for an exponential relationship).

-Remove Noise. Linear regression assumes that your input and output variables are not noisy. Consider using data cleaning operations that let you better expose and clarify the signal in your data. This is most important for the output variable and you want to remove outliers in the output variable (y) if possible.

-Remove Collinearity. Linear regression will over-fit your data when you have highly correlated input variables. Consider calculating pairwise correlations for your input data and removing the most correlated.

-Gaussian Distributions. Linear regression will make more reliable predictions if your input and output variables have a Gaussian distribution. You may get some benefit using transforms (e.g. log or BoxCox) on you variables to make their distribution more Gaussian looking.

-Rescale Inputs: Linear regression will often make more reliable predictions if you rescale input variables using standardization or normalization.

# 1. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train,y_train)

# Model Evaluation
Let's evaluate the model by checking out it's coefficients and how we can interpret them.

In [ ]:
# print the intercept
print(lin_reg.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lin_reg.coef_, X.columns, columns=['Coefficient'])
coeff_df

Interpreting the coefficients:

* Holding all other features fixed, a 1 unit increase in Avg. Area Income is associated with an increase of $21.52.

* Holding all other features fixed, a 1 unit increase in Avg. Area House Age is associated with an increase of $164883.28.

* Holding all other features fixed, a 1 unit increase in Avg. Area Number of Rooms is associated with an increase of $122368.67.

* Holding all other features fixed, a 1 unit increase in Avg. Area Number of Bedrooms is associated with an increase of $2233.80.

* Holding all other features fixed, a 1 unit increase in Area Population is associated with an increase of $15.15.

# Predictions from our Model
So Let's go ahead and do some predictions off our test set and see how well it do!

In [ ]:
pred = lin_reg.predict(X_test)

Here we Plot the 2 entities : Observed V/S Predicted Values using scatter plot and see how close 2 are.

In [ ]:
plt.scatter(y_test, pred)

# Residual Histogram
Residuals = Observed value - Predicted Value which is (y_test - pred) in our case here.
So we see the distribution of the residuals using distribution plot down below.

In [ ]:
sns.distplot((y_test - pred), bins=50);

Regression Evaluation Metrics
Here are three common evaluation metrics for regression problems:

Mean Absolute Error (MAE) is the mean of the absolute value of the errors:

$$
1/n∑i=1/n|y{i}−y^{i}|
$$ 

Mean Squared Error (MSE) is the mean of the squared errors:

$$
1/n∑i=1/n(yi−y^i)^2
$$ 

Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors:

$$
1/n∑i=√1/n(yi−y^i)^2
$$ 

Comparing these metrics:

MAE is the easiest to understand, because it's the average error.
MSE is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
RMSE is even more popular than MSE, because RMSE is interpretable in the "y" units.
All of these are loss functions, because we want to minimize them.

In [ ]:
print_evaluate(y_test, lin_reg.predict(X_test))

In [ ]:
results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df

# 2. Robust Regression

Robust regression is a form of regression analysis designed to overcome some limitations of traditional parametric and non-parametric methods. Robust regression methods are designed to be not overly affected by violations of assumptions by the underlying data-generating process.

One instance in which robust estimation should be considered is when there is a strong suspicion of heteroscedasticity.

A common situation in which robust estimation is used occurs when the data contain outliers. In the presence of outliers that do not come from the same data-generating process as the rest of the data, least squares estimation is inefficient and can be biased. Because the least squares predictions are dragged towards the outliers, and because the variance of the estimates is artificially inflated, the result is that outliers can be masked. (In many situations, including some areas of geostatistics and medical statistics, it is precisely the outliers that are of interest.)

# Random Sample Consensus - RANSAC

Random Sample Consensus - RANSAC
Random sample consensus (RANSAC) is an iterative method to estimate parameters of a mathematical model from a set of observed data that contains outliers, when outliers are to be accorded no influence on the values of the estimates. Therefore, it also can be interpreted as an outlier detection method.

A basic assumption is that the data consists of "inliers", i.e., data whose distribution can be explained by some set of model parameters, though may be subject to noise, and "outliers" which are data that do not fit the model. The outliers can come, for example, from extreme values of the noise or from erroneous measurements or incorrect hypotheses about the interpretation of data. RANSAC also assumes that, given a (usually small) set of inliers, there exists a procedure which can estimate the parameters of a model that optimally explains or fits this data.

In [ ]:
from sklearn.linear_model import RANSACRegressor

model = RANSACRegressor()
model.fit(X_train, y_train)

pred = model.predict(X_test)
print_evaluate(y_test, pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Robust Regression", *evaluate(y_test, pred) , cross_val(RANSACRegressor())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# 3. Ridge Regression

Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of coefficients. The ridge coefficients minimize a penalized residual sum of squares,

$ minw∣∣∣∣Xw−y∣∣∣∣22+α∣∣∣∣w∣∣∣∣22 $
 
α>=0  is a complexity parameter that controls the amount of shrinkage: the larger the value of  α , the greater the amount of shrinkage and thus the coefficients become more robust to collinearity.

Ridge regression is an L2 penalized model. Add the squared sum of the weights to the least-squares cost function.

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(X_train, y_train)
pred = model.predict(X_test)

print_evaluate(y_test, pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Ridge Regression", *evaluate(y_test, pred) , cross_val(Ridge())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# 4. LASSO Regression

A linear model that estimates sparse coefficients.

Mathematically, it consists of a linear model trained with  ℓ1  prior as regularizer. The objective function to minimize is:

$ minw12nsamples∣∣∣∣Xw−y∣∣∣∣22+α∣∣∣∣w∣∣∣∣1 $
 
The lasso estimate thus solves the minimization of the least-squares penalty with  α∣∣∣∣w∣∣∣∣1  added, where  α  is a constant and  ∣∣∣∣w∣∣∣∣1  is the  ℓ1−norm  of the parameter vector.

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print_evaluate(y_test,pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Lasso Regression", *evaluate(y_test, pred) , cross_val(Lasso())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# 5. Elastic Net

A linear regression model trained with L1 and L2 prior as regularizer.

This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.

Elastic-net is useful when there are multiple features which are correlated with one another. Lasso is likely to pick one of these at random, while elastic-net is likely to pick both.

A practical advantage of trading-off between Lasso and Ridge is it allows Elastic-Net to inherit some of Ridge’s stability under rotation.

The objective function to minimize is in this case

$ minw12nsamples∣∣∣∣Xw−y∣∣∣∣22+αρ∣∣∣∣w∣∣∣∣1+α(1−ρ)2∣∣∣∣w∣∣∣∣22 $

In [ ]:
from sklearn.linear_model import ElasticNet

model = ElasticNet()
model.fit(X_train,y_train)
model.predict(X_test)

print_evaluate(y_test,pred)



In [ ]:
results_df_2 = pd.DataFrame(data=[["Elastic Net Regression", *evaluate(y_test, pred) , cross_val(ElasticNet())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# 6. Polynomial Regression

One common pattern within machine learning is to use linear models trained on nonlinear functions of the data. This approach maintains the generally fast performance of linear methods, while allowing them to fit a much wider range of data.

For example, a simple linear regression can be extended by constructing polynomial features from the coefficients. In the standard linear regression case, you might have a model that looks like this for two-dimensional data:

$ y^(w,x)=w0+w1x1+w2x2 $
 
If we want to fit a paraboloid to the data instead of a plane, we can combine the features in second-order polynomials, so that the model looks like this:

$ y^(w,x)=w0+w1x1+w2x2+w3x1x2+w4x21+w5x22 $
 
The (sometimes surprising) observation is that this is still a linear model: to see this, imagine creating a new variable

$ z=[x1,x2,x1x2,x21,x22] $
 
With this re-labeling of the data, our problem can be written

$ y^(w,x)=w0+w1z1+w2z2+w3z3+w4z4+w5z5 $
 
We see that the resulting polynomial regression is in the same class of linear models we’d considered above (i.e. the model is linear in w) and can be solved by the same techniques. By considering linear fits within a higher-dimensional space built with these basis functions, the model has the flexibility to fit a much broader range of data.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree=2)
X_poly = poly_reg.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.4, random_state=101)

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train,y_train)
pred = lin_reg.predict(X_test)

print_evaluate(y_test, pred)


In [ ]:
results_df_2 = pd.DataFrame(data=[["Polynomail Regression", *evaluate(y_test, pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# 7. Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor()
model.fit(X_train, y_train)

pred = model.predict(X_test)

# Scoring the Random Forest Regression

print_evaluate(y_test,pred)

random_r2_score = r2_score(model.predict(X_test), y_test)
random_r2_score

In [ ]:
results_df_2 = pd.DataFrame(data=[["Random Forest Regressor", *evaluate(y_test, pred),cross_val(RandomForestRegressor()) ]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# 8. Decision Tree Model

In [ ]:
from sklearn import tree

model = tree.DecisionTreeRegressor()

model.fit(X_train,y_train)

pred = model.predict(X_test)

print_evaluate(y_test,pred)

# checking R2 score

tree_r2_score = r2_score(model.predict(X_test),y_test)
tree_r2_score


In [ ]:
results_df_2 = pd.DataFrame(data=[["Decision Tree Regression", *evaluate(y_test, pred), cross_val(tree.DecisionTreeRegressor())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

# Summary

In this notebook you discovered the linear regression algorithm for machine learning.

You covered a lot of ground including:

The common linear regression models (Ridge, Lasso, ElasticNet, ...).
The representation used by the model.
Learning algorithms used to estimate the coefficients in the model.
Rules of thumb to consider when preparing data for use with linear regression.
How to evaluate a linear regression model.